### 1. IMPORTACIÓN DE LIBRERÍAS

In [13]:
import pandas as pd
import numpy as np

### 2. CARGA DE LOS DATOS (DE MOMENTO CONSIDERANDO UN .CSV)

In [14]:
def load_data(file_path: str) -> pd.DataFrame | None:
    """
    Carga un conjunto de datos desde una ruta de archivo CSV.

    Esta función encapsula la lógica de lectura de datos con Pandas,
    incluyendo un manejo básico de errores si el archivo no se encuentra.

    Args:
        file_path (str): La ruta al archivo .csv que se va a cargar.

    Returns:
        pd.DataFrame | None: Un DataFrame de Pandas con los datos cargados,
        o None si ocurre un error (ej. archivo no encontrado).
    """
    try:
        # Intenta leer el archivo CSV y lo carga en un DataFrame
        df = pd.read_csv(file_path)
        print(f"Datos cargados exitosamente desde: {file_path}")
        return df
    except FileNotFoundError:
        # Manejo de error si el archivo no existe en la ruta especificada
        print(f"Error: El archivo no fue encontrado en la ruta: {file_path}")
        return None
    except Exception as e:
        # Manejo de otros posibles errores durante la carga
        print(f"Ocurrió un error inesperado al cargar el archivo: {e}")
        return None

### 3. DIAGNÓSTICO INICIAL

In [15]:
def get_data_overview(df: pd.DataFrame, df_name: str = "DataFrame") -> None:
    """
    Imprime un resumen completo y diagnóstico de un DataFrame.

    Incluye dimensiones, tipos de datos, estadísticas descriptivas,
    conteo de duplicados y porcentaje de valores nulos.

    Args:
        df (pd.DataFrame): El DataFrame que se va a analizar.
        df_name (str): Un nombre opcional para el DataFrame que se mostrará en los reportes.
    """
    print(f"\n===============ANÁLISIS EXPLORATORIO RÁPIDO PARA: '{df_name}'===============")

    # 1. Dimensiones del DataFrame
    print(f"\n**Dimensiones:** {df.shape[0]} filas y {df.shape[1]} columnas.")

    # 2. Tipos de datos y valores no nulos
    print("\n**Tipos de Datos y Valores No Nulos:**")
    df.info()

    # 3. Estadísticas Descriptivas para variables numéricas
    print("\n**Estadísticas Descriptivas (Numéricas):**")
    # Usamos .T para transponer la tabla y hacerla más legible
    print(df.describe().T)

    # 4. Conteo de filas duplicadas
    duplicates = df.duplicated().sum()
    print(f"\n**Filas Duplicadas:** {duplicates} filas duplicadas encontradas.")

    # 5. Porcentaje de Valores Nulos por columna
    null_percentage = (df.isnull().sum() / len(df)) * 100
    null_info = null_percentage[null_percentage > 0].sort_values(ascending=False)
    
    if not null_info.empty:
        print("\n**Porcentaje de Valores Nulos (>0%):**")
        print(null_info)
    else:
        print("\n**No se encontraron valores nulos.**")
    
    print("\nFin del análisis.")

### 4. LIMPIEZA DE LOS DATOS

##### Validación del esquema. Eliminación de columnas no identificadas

In [16]:
def drop_unnecessary_columns(df: pd.DataFrame, valid_columns: list) -> pd.DataFrame:
    """
    Elimina las columnas de un DataFrame que no están en una lista de columnas válidas.

    Esta función es útil para asegurar que el DataFrame solo contenga las columnas
    esperadas según el esquema definido.

    Args:
        df (pd.DataFrame): El DataFrame a limpiar.
        valid_columns (list): Una lista de strings con los nombres de las
                              columnas que deben permanecer.

    Returns:
        pd.DataFrame: Un nuevo DataFrame que solo contiene las columnas válidas.
    """

    # Identifica las columnas actuales del DataFrame
    current_columns = df.columns.tolist()
    
    # Encuentra las columnas que están en el DataFrame pero no en la lista de válidas
    cols_to_drop = [col for col in current_columns if col not in valid_columns]

    if cols_to_drop:
        print(f"\nColumnas a eliminar: {cols_to_drop}")
        # Elimina las columnas identificadas y devuelve una copia del df modificado
        df_cleaned = df.drop(columns=cols_to_drop)
        print("Columnas innecesarias eliminadas.")
    else:
        print("No se encontraron columnas innecesarias. El esquema es correcto.")
        df_cleaned = df.copy() # Devuelve una copia para mantener la consistencia

    return df_cleaned

##### Conversión del tipo correcto de columnas

In [17]:
def correct_initial_data_types(df: pd.DataFrame) -> pd.DataFrame:
    """
    Corrige los tipos de datos de columnas específicas a numérico y fecha.
    Las categóricas de momento igual se consideran como numéricas, para facilitar el manejo de inválidos.

    Usa 'errors=coerce' para convertir valores no válidos en NaN,
    facilitando su manejo posterior.

    Args:
        df (pd.DataFrame): El DataFrame a procesar.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con los tipos de datos corregidos.
    """

    df_corrected = df.copy()
    print("\nIniciando corrección de tipos de datos...")

    # Define los grupos de columnas
    numeric_cols = ['temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered', 'cnt',
                    'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
    
    # 1. Procesa columnas numéricas
    for col in numeric_cols:
        # Omite si la columna no existe (por si fue eliminada antes)
        if col in df_corrected.columns:
            df_corrected[col] = pd.to_numeric(df_corrected[col], errors='coerce')

    # 2. Procesa columna de fecha
    if 'dteday' in df_corrected.columns:
        # Usamos format='mixed' para manejar explícitamente los formatos múltiples
        df_corrected['dteday'] = pd.to_datetime(
            df_corrected['dteday'], 
            errors='coerce', 
            format='mixed' # Esta es la solución
        )
        
    print("Tipos de datos corregidos de forma semántica.")
    return df_corrected

##### Manejo de valores no válidos

In [18]:
def handle_invalid_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Valida los datos contra un conjunto de reglas y convierte los inválidos a NaN.

    Args:
        df (pd.DataFrame): El DataFrame con tipos de datos ya corregidos.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con los valores inválidos convertidos a NaN.
    """

    df_validated = df.copy()
    print("\nIniciando validación de valores...")

    # Diccionario de reglas de validación
    # Para categóricas: lista de valores permitidos
    # Para numéricas: tupla con (valor_mínimo, valor_máximo)
    validation_rules = {
        'dteday': (pd.to_datetime('2011-01-01'), pd.to_datetime('2012-12-31')),
        'season': [1, 2, 3, 4],
        'yr': [0, 1],
        'mnth': list(range(1, 13)),
        'hr': list(range(0, 24)),
        'holiday': [0, 1],
        'weekday': list(range(0, 7)),
        'workingday': [0, 1],
        'weathersit': [1, 2, 3, 4],
        'hum': (0.0, 1.0),
        'windspeed': (0.0, 1.0),
        'cnt': (0, float('inf')) # El conteo no puede ser negativo
    }

    for column, rule in validation_rules.items():
        if column in df_validated.columns:
            # Conteo inicial de nulos para reporte
            initial_nulls = df_validated[column].isnull().sum()

            # Validación para variables categóricas (regla es una lista)
            if isinstance(rule, list):
                invalid_mask = ~df_validated[column].isin(rule)
            # Validación para variables numéricas (regla es una tupla)
            elif isinstance(rule, tuple):
                min_val, max_val = rule
                invalid_mask = (df_validated[column] < min_val) | (df_validated[column] > max_val)
            
            # Reemplaza los valores que no cumplen la regla con NaN
            df_validated.loc[invalid_mask, column] = np.nan
            
            # Reporta cuántos valores inválidos se encontraron y corrigieron
            final_nulls = df_validated[column].isnull().sum()
            newly_invalid = final_nulls - initial_nulls
            if newly_invalid > 0:
                print(f"  -> Columna '{column}': {newly_invalid} valores inválidos convertidos a NaN.")

    print("Validación de valores completada.")
    return df_validated